In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

import cloudinary

import cloudinary.api

In [32]:
import os
from pathlib import Path
from dotenv import load_dotenv

project_root = Path.cwd().parent
load_dotenv(project_root / '.env')

cloud_api_key = os.getenv('CLOUD_API_KEY')
cloud_api_secret = os.getenv('CLOUD_API_SECRET')
cloud_name = 'quinn-cloudinary'


In [ ]:
import cloudinary
import cloudinary.api
from pathlib import Path
import os

class CloudinaryHandler:
    def __init__(self, cloud_name: str, api_key: str, api_secret: str):
        """
        Initialize CloudinaryHandler with credentials.
        
        Args:
            cloud_name: Cloudinary cloud name
            api_key: Cloudinary API key
            api_secret: Cloudinary API secret
        """
        self.cloud_name = cloud_name
        self.api_key = api_key
        self.api_secret = api_secret
        self._configure_cloudinary()
        
    def _configure_cloudinary(self):
        """Configure Cloudinary with the provided credentials."""
        cloudinary.config(
            cloud_name=self.cloud_name,
            api_key=self.api_key,
            api_secret=self.api_secret,
            secure=True
        )
    
    def _change_mp4_format(self, url: str) -> str:
        """Convert MP4 URL to MP3 format."""
        return url.replace('.mp4', '.mp3')
    
    def pull_audio_details(self, max_results: int = 10) -> dict:
        """
        Fetches video resources from Cloudinary and converts their URLs to MP3 format.
        
        Args:
            max_results: Maximum number of video resources to fetch.
            
        Returns:
            A dictionary of Asset objects with modified MP3 URLs.
        """
        resp = cloudinary.api.resources(resource_type="video", max_results=max_results)
        asset_dict = {}
        
        audio_dir = Path("app/static/audio")
        audio_dir.mkdir(parents=True, exist_ok=True)
        
        
        for resource in resp["resources"]:
            filename = f"{resource['asset_id']}.mp3"
            file_path = audio_dir / filename
            
            formatted_url = self._change_mp4_format(resource["secure_url"])
            
            asset_dict[resource["asset_id"]] = {
                "public_id": resource["public_id"],
                "asset_id": resource["asset_id"],
                "secure_url": formatted_url,
                "audio_path": file_path,
                "captions": None
            }
            
        return asset_dict


handler = CloudinaryHandler(
    cloud_name=cloud_name,
    api_key=cloud_api_key,
    api_secret=cloud_api_secret
)

audio_details = handler.pull_audio_details()
print(f"Found {len(audio_details)} audio resources")
for asset_id, details in audio_details.items():
    print(f"\nAsset ID: {asset_id}")
    print(f"Public ID: {details['public_id']}")
    print(f"URL: {details['secure_url']}")
    print(f"Local path: {details['audio_path']}")
